In [3]:
from datetime import datetime, date

In [72]:
#92 ou 184 jours 60/92 ou 120/184  travaille ou a travaille
#Pour gaeg dn_horairehebdosalairie>=17.5
#Pour DSN ragarder dc_modalitetpstravail_id 
# Les nouvelles prestation sont ACL==Active emploi ex AEM et EMS pour les permittants
#Pour acl on prends les contrat>=1 mois
dc_modalitetpstravail_id=10 on prend, sinon 
#Contrat.UniteMesure  Contrat.UniteMesure : dc_unitequotitetravail_id
10 - heure
12 - journée
20 - forfait jour
21 - forfait heure
31 - à la pige
32 - à la vacation
33 - à la tâche
90 - salarié sans unité de mesure définie au contrat
99 - salarié non concerné
page 142

pour DSN >78 heure par mois pris en compte dc_unitequotitetravail_id ,dn_quotitetravail , dc_modalitetpstravail_id  ,

In [ ]:
Contrat.ModaliteTemps = dc_modalitetpstravail_id
10 - temps plein
20 - temps partiel
21 - temps partiel thérapeutique
99 - salarié non concerné

## Identification des bénéficiaire des prestation AEM 
* On détermine les bénéficiare de prestation AEM
* On détermine le résultat de la prestation
* On cherche le NIR des bénéficiaires des prestations

In [94]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
commandepresta = dataiku.Dataset("commandepresta")
pr00_ppx005_drpgep_commandepresta = commandepresta.get_dataframe()

commandebeneficiaire = dataiku.Dataset("commandebeneficiaire")
pr00_ppx005_drpgep_commandebeneficiaire = commandebeneficiaire.get_dataframe()

commanderesultat = dataiku.Dataset("commanderesultat")
pr00_ppx005_drpgep_commanderesultat = commanderesultat.get_dataframe()

refh_individu = dataiku.Dataset("refh_individuNIRbis")
pr00_ppx005_refh_individu = refh_individu.get_dataframe()


#ref_resultatbeneficiaire = dataiku.Dataset("ref_resultatbeneficiaire")
#pr00_ppx005_ref_resultatbeneficiaire = ref_resultatbeneficiaire.get_dataframe()

#xdpdpa_dpae_new = dataiku.Dataset("xdpdpa_dpae_new")
#pr00_ppx005_xdpdpa_dpae = xdpdpa_dpae_new.get_dataframe()

#dsnpe_identitedeclareesal = dataiku.Dataset("dsnpe_identitedeclareesal")
#pr00_pdsnpe_identitedeclareesal = dsnpe_identitedeclareesal.get_dataframe()

#dsnpe_declaractivitesal = dataiku.Dataset("dsnpe_declaractivitesal")
#pr00_pdsnpe_declaractivitesal = dsnpe_declaractivitesal.get_dataframe()

#drvder_periodeactivitegaec = dataiku.Dataset("drvder_periodeactivitegaec")
#pr00_ppx005_drvder_periodeactivitegaec = drvder_periodeactivitegaec.get_dataframe()


# Write recipe outputs
#beneficiaireeaem = dataiku.Dataset("BENEFICIAIREEAEM")
#beneficiaireeaem.write_with_schema(beneficiaireeaem_df)

In [5]:
pr00_ppx005_drpgep_commanderesultat.shape

(17790, 25)

#### On détermine les commandes de prestation. Ceci donne les dates des prestations 

In [95]:
#On détermine les commande de prestation
commandepresta = pr00_ppx005_drpgep_commandepresta.loc[(pr00_ppx005_drpgep_commandepresta['dc_typepresta_id']=='AEM')\
                                                       & (pr00_ppx005_drpgep_commandepresta['dd_finprevisionnelle']>=datetime(2019,12,10))\
                                                       & (pr00_ppx005_drpgep_commandepresta['dd_finprevisionnelle']<=datetime(2019,12,30))\
                                                       & (pr00_ppx005_drpgep_commandepresta['dc_statutworkflow_id'].isin(['DPE','SOM','ECT']))]
commandepresta = commandepresta.drop_duplicates(subset=['dc_codesiteregion_id','dc_typepresta_id','dc_commandepresta','kn_idcommandepresta'], keep='last')

columns_to_drop = ['dd_datecreationcommande','dc_statutworkflow_id','dc_drorigine','dc_prestatairerealisation','yearraff','monthraff']

commandepresta = commandepresta.drop(columns_to_drop,axis=1)

#### On détermine les bénificiaires des prestations en lien avec les commandes

In [96]:
#On enlève les doublons des bénéficiaires de prestation
pr00_ppx005_drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire.drop_duplicates(subset=['dc_codesiteregion_id','dc_typepresta_id','dc_commandepresta','kn_idcommandepresta','kn_commandebeneficiaire','dn_individu_national'], keep='last')


pr00_ppx005_drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire.drop(['dc_dispositif_id', 'dd_debutdispositif'], axis=1)

#On fait une jointure avec les commandes de prestation
pr00_ppx005_drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire.merge(commandepresta,\
                                                                                        left_on=['dc_codesiteregion_id','dc_typepresta_id','dc_commandepresta','kn_idcommandepresta'],\
                                                                                        right_on=['dc_codesiteregion_id','dc_typepresta_id','dc_commandepresta','kn_idcommandepresta'],how='inner')

In [7]:
pr00_ppx005_drpgep_commandebeneficiaire.shape

(2965, 9)

#### Les résultats des prestations 

In [97]:
pr00_ppx005_drpgep_commanderesultat = pr00_ppx005_drpgep_commanderesultat.drop_duplicates(subset=['dc_codesiteregion_id','dc_typepresta_id', 'dc_commandepresta', 'kn_idcommandepresta','kn_commandebeneficiaire_id'],keep='last')

columns_to_keep = ['dc_codesiteregion_id','dc_typepresta_id','dc_commandepresta','kn_idcommandepresta','kn_commandebeneficiaire_id','kc_resultatsbeneficiaire_id', 'dd_dateeffetresultat']

pr00_ppx005_drpgep_commanderesultat = pr00_ppx005_drpgep_commanderesultat[columns_to_keep]

#jointure interne entre pr00_ppx005_drpgep_commandebeneficiaire et pr00_ppx005_drpgep_commanderesultat 
pr00_ppx005_drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire.merge(\
                                                                                        pr00_ppx005_drpgep_commanderesultat,how='inner',\
                                                                                        left_on=['dc_codesiteregion_id','dc_commandepresta','kn_commandebeneficiaire'],\
                                                                                        right_on=['dc_codesiteregion_id','dc_commandepresta','kn_commandebeneficiaire_id'])

In [98]:
resultat = {'VA2':"SORTIE ANTICIPEE/ABANDON A PARTIR DU 2EME MOIS",\
            'VAB':"SORTIE ANTICIPEE/ABANDON AVANT LE 2EME MOIS",\
            'VAT':"A SUIVI LA PRESTATION DANS SA TOTALITE",\
            'VED':"REPRISE D'EMPLOI CDD OU MISSION INTERIM DE 3 A 6 MOIS",\
            'VEM':"REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM >= A 6 MOIS"}

pr00_ppx005_drpgep_commandebeneficiaire['resultat_libele'] = pr00_ppx005_drpgep_commandebeneficiaire['kc_resultatsbeneficiaire_id'].apply(lambda x:resultat[x])


In [10]:
list(pr00_ppx005_refh_individu)

['td_datedebut', 'kn_individu_national', 'dc_individu_local', 'nir']

#### On cherche les NIR des bénéficiaires pour pouvoir faire des jointure avec la DSN
Attention les NIR doivent être de taille 13 

In [99]:
#On filtre la table pr00_ppx005_refh_individu :
list_BNI = list(pr00_ppx005_drpgep_commandebeneficiaire['dn_individu_national'])

pr00_ppx005_refh_individu = pr00_ppx005_refh_individu.loc[pr00_ppx005_refh_individu['kn_individu_national'].isin(list_BNI)]

#pr00_ppx005_refh_individu = pr00_ppx005_refh_individu.drop_duplicates(subset=['kn_individu_national','dc_nir'],keep='last')

list_nir = list(set(pr00_ppx005_refh_individu['nir']))

#group by 'kn_individu_national' puis on joint avec beneficiare formation

In [12]:
#On eput enlever dc_typepresta_id_y ,kn_idcommandepresta_y, kn_commandebeneficiaire_id
#pr00_ppx005_drpgep_commandebeneficiaire.head()

## Les contrats des DPAE
* Les BNI des DPAE doivent être ceux de la table pr00_ppx005_drpgep_commandebeneficiaire
* Les dates de début de contrat sont superieur ou égale au début de la prestation

In [100]:
#On ramene les contratq de la DPAE 
xdpdpa_dpae_new = dataiku.Dataset("xdpdpa_dpae_new")
pr00_ppx005_xdpdpa_dpae = xdpdpa_dpae_new.get_dataframe()

list_BNI = list(pr00_ppx005_drpgep_commandebeneficiaire['dn_individu_national'])

pr00_ppx005_xdpdpa_dpae = pr00_ppx005_xdpdpa_dpae.loc[pr00_ppx005_xdpdpa_dpae['dc_individu_national'].isin(list_BNI)]

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [101]:
pr00_ppx005_xdpdpa_dpae = pr00_ppx005_xdpdpa_dpae.merge(pr00_ppx005_drpgep_commandebeneficiaire[['dn_individu_national','dd_datedebut','dd_finprevisionnelle','kc_resultatsbeneficiaire_id']]\
                                                        ,left_on=['dc_individu_national'], right_on=['dn_individu_national'],\
                                                        how='inner')

#On calcule la date de fin d'observation
from datetime import timedelta
pr00_ppx005_xdpdpa_dpae['date_fin_observation'] = pr00_ppx005_xdpdpa_dpae['dd_datedebut'].apply(lambda x:x+timedelta(days=5*30))

In [102]:
#On selectionne les contrat entre le debut de la prestation et 5 mois plus tard 
pr00_ppx005_xdpdpa_dpae = pr00_ppx005_xdpdpa_dpae.loc[pr00_ppx005_xdpdpa_dpae['kd_dateembauche']>=pr00_ppx005_xdpdpa_dpae['dd_datedebut']]
pr00_ppx005_xdpdpa_dpae = pr00_ppx005_xdpdpa_dpae.loc[pr00_ppx005_xdpdpa_dpae['kd_dateembauche']<=pr00_ppx005_xdpdpa_dpae['date_fin_observation']]

#### On enlève les mission intérim des DPAE car de mauvaise qualité

In [103]:
#pr00_ppx005_xdpdpa_dpae.head(2)
import numpy as np
conditions = [pr00_ppx005_xdpdpa_dpae['dc_typecontrat_id']==1.0,\
             pr00_ppx005_xdpdpa_dpae['dc_typecontrat_id']==2.0,\
             pr00_ppx005_xdpdpa_dpae['dc_typecontrat_id']==3.0]

values = ['CDD','CDI','CTT']
pr00_ppx005_xdpdpa_dpae['type_contrat'] = np.select(conditions, values, 'NR')

#on enleve les CTT
pr00_ppx005_xdpdpa_dpae = pr00_ppx005_xdpdpa_dpae.loc[pr00_ppx005_xdpdpa_dpae['type_contrat'].isin(['CDD','CDI'])]


from datetime import timedelta, datetime
import calendar

pr00_ppx005_xdpdpa_dpae['kd_dateembauche_7avant'] = pr00_ppx005_xdpdpa_dpae['kd_dateembauche'].apply(lambda x:x-timedelta(days=7))

pr00_ppx005_xdpdpa_dpae['kd_dateembauche_mois_suiv'] = pr00_ppx005_xdpdpa_dpae['kd_dateembauche'].apply(lambda x:datetime(x.year,x.month+1,calendar.monthrange(x.year,x.month+1)[1]) if x.month!=12 else datetime(x.year+1,1,31))

In [224]:
list(pr00_ppx005_xdpdpa_dpae)

['td_datmaj',
 'kc_moisstatistique',
 'kd_datecreation',
 'kc_siret',
 'dc_naf_id',
 'dc_nomraisonsociale',
 'dc_codepostal',
 'dn_employeur',
 'kc_nir',
 'kd_datenaissance',
 'dc_individu_local',
 'dc_individu_national',
 'kd_dateembauche',
 'dc_typecontrat_id',
 'dd_datefincdd',
 'dc_unite',
 'dn_individu_national',
 'dd_datedebut',
 'dd_finprevisionnelle',
 'kc_resultatsbeneficiaire_id',
 'date_fin_observation',
 'type_contrat',
 'kd_dateembauche_7avant',
 'kd_dateembauche_mois_suiv']

## La table des prises en charge pr00_ppx005_ddiadh_priseencharge sert à enrichire les DPAE. 
* Les jointures se font sur des BNI et des date d'actualisation comparé à des dates d'embauche
* Les DPAE de type CDI sont concernées par cette enrichissement

In [104]:
ppx005_ddiadh_priseencharge = dataiku.Dataset("ppx005_ddiadh_priseencharge")
pr00_ppx005_ddiadh_priseencharge = ppx005_ddiadh_priseencharge.get_dataframe()

list_BNI = list(pr00_ppx005_drpgep_commandebeneficiaire['dn_individu_national'])

pr00_ppx005_ddiadh_priseencharge = pr00_ppx005_ddiadh_priseencharge.loc[pr00_ppx005_ddiadh_priseencharge['kn_individu_national'].isin(list_BNI)]

dc_motifcessationpec_keep = ['11','12','13','14','15','20','21','22','23','24','25','90']
pr00_ppx005_ddiadh_priseencharge = pr00_ppx005_ddiadh_priseencharge.loc[pr00_ppx005_ddiadh_priseencharge['dc_motifcessationpec_id'].isin(dc_motifcessationpec_keep)]

In [43]:
#list(pr00_ppx005_ddiadh_priseencharge)
#pr00_ppx005_ddiadh_priseencharge.drop_duplicates(subset=['kn_individu_national','dc_assedicde','kn_numpec','dc_typepec_id'],keep='last').shape
#pr00_ppx005_ddiadh_priseencharge.shape

#### On prend les évènements de prise en charge compris entre la date de dédbut de la prestation et la date de fin d'observation
date fin observation = date début prestation+5 mois

In [105]:
pr00_ppx005_ddiadh_priseencharge = pr00_ppx005_ddiadh_priseencharge.merge(pr00_ppx005_drpgep_commandebeneficiaire[['dn_individu_national','dd_datedebut','dd_finprevisionnelle','kc_resultatsbeneficiaire_id']]\
                                                                          ,left_on=['kn_individu_national'],right_on=['dn_individu_national'],\
                                                                          how='inner')
                                                        
#On calcule la date de fin d'observation
from datetime import timedelta
pr00_ppx005_ddiadh_priseencharge['date_fin_observation'] = pr00_ppx005_ddiadh_priseencharge['dd_datedebut'].apply(lambda x:x+timedelta(days=8*30))

pr00_ppx005_ddiadh_priseencharge = pr00_ppx005_ddiadh_priseencharge.loc[pr00_ppx005_ddiadh_priseencharge['dd_dateannulationpec']>=pr00_ppx005_ddiadh_priseencharge['dd_datedebut']]
pr00_ppx005_ddiadh_priseencharge = pr00_ppx005_ddiadh_priseencharge.loc[pr00_ppx005_ddiadh_priseencharge['dd_dateannulationpec']<=pr00_ppx005_ddiadh_priseencharge['date_fin_observation']]

pr00_ppx005_ddiadh_priseencharge = pr00_ppx005_ddiadh_priseencharge.drop_duplicates(subset=['kn_individu_national','dc_assedicde','kn_numpec','dc_typepec_id'],keep='last')

In [106]:

list(pr00_ppx005_xdpdpa_dpae)
var_prise_en_charge = ['kn_individu_national','kc_individu_local','kn_numpec','dc_typepec_id','dd_dateannulationpec']
prise_en_charge = pr00_ppx005_ddiadh_priseencharge[var_prise_en_charge]
pr00_ppx005_xdpdpa_dpaebis = pr00_ppx005_xdpdpa_dpae.merge(prise_en_charge,left_on=['dc_individu_national'],right_on=['kn_individu_national'], how='inner')


#pr00_ppx005_xdpdpa_dpaebis.head()
pr00_ppx005_xdpdpa_dpaebis = pr00_ppx005_xdpdpa_dpaebis.loc[pr00_ppx005_xdpdpa_dpaebis['dd_dateannulationpec']>=pr00_ppx005_xdpdpa_dpaebis['kd_dateembauche_7avant']]
pr00_ppx005_xdpdpa_dpaebis = pr00_ppx005_xdpdpa_dpaebis.loc[pr00_ppx005_xdpdpa_dpaebis['dd_dateannulationpec']<=pr00_ppx005_xdpdpa_dpaebis['kd_dateembauche_mois_suiv']]
#pr00_ppx005_xdpdpa_dpaebis.groupby(['dc_individu_national',]).


pr00_ppx005_xdpdpa_dpaebis['pec'] = pr00_ppx005_xdpdpa_dpaebis[['kn_numpec','dc_typepec_id','dd_dateannulationpec']].apply(lambda x:(x[0],x[1],x[2]),axis=1)

pec = pd.DataFrame(pr00_ppx005_xdpdpa_dpaebis.groupby(['kn_individu_national','kd_dateembauche'])['pec'].apply(lambda x: list(x))).reset_index()
                   
                   

In [107]:
#hh = {'num':[1,2,1,2,1,2,1,2],'string':['prem','deux','trois','quatre','5','six','sept','huit']}
#hhbis = pd.DataFrame(hh)
#hhbis.groupby('num')['string'].apply(lambda x: ', '.join(x))
pec = pd.DataFrame(pr00_ppx005_xdpdpa_dpaebis.groupby(['kn_individu_national','kd_dateembauche'])['pec'].apply(lambda x: list(x))).reset_index()
pec.head(2)

,kn_individu_national,kd_dateembauche,pec
0,1002965535,2020-01-13,"[(40, 1, 2020-01-13 00:00:00)]"
1,1025963674,2020-01-08,"[(30, 1, 2020-01-31 00:00:00)]"


#### Ligne gaec de type 11=Activité déclarée travail et d'origine RG et A8 A10 (61 62). On garde si 78>=dn_quantiteactivite

In [108]:
#ppx005_drvder_periodegaec_dpae
ppx005_drvder_periodegaec_dpae = dataiku.Dataset("ppx005_drvder_periodegaec_dpae")
pr00_ppx005_drvder_periodeactivitegaec = ppx005_drvder_periodegaec_dpae.get_dataframe()

In [109]:
list_BNI = list(pr00_ppx005_drpgep_commandebeneficiaire['dn_individu_national'])

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['kn_individu_national'].isin(list_BNI)]

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dc_origineinformationgaec_id'].isin([61,62])]

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dc_typeperiodegaec_id'].isin([11])]
#`dc_origineinformationgaec_id` IN ('61','62')
# AND `dc_typeperiodegaec_id`='11'

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dn_quantiteactivite']>=78]
      
pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.drop_duplicates(subset=['kn_individu_national','kn_periodeactivitegaec'],keep='last')  

#colonne mois statistique
pr00_ppx005_drvder_periodeactivitegaec['MOISTAT1'] = pr00_ppx005_drvder_periodeactivitegaec['dd_datedebutactivite'].apply(lambda x:str(x.year)+str(x.month))

In [ ]:
list()

In [110]:
#Jointure entre les 
#list(pr00_ppx00#5_drvder_periodeactivitegaec)

var_gaec = ['kn_periodeactivitegaec','kn_individu_national','MOISTAT1','dn_quantiteactivite','dc_uniteactivite_id']
gaec = pr00_ppx005_drvder_periodeactivitegaec[var_gaec]
pr00_ppx005_xdpdpa_dpaetier = pr00_ppx005_xdpdpa_dpae.merge(gaec,left_on=['dc_individu_national'],right_on=['kn_individu_national'], how='inner')

In [287]:
pr00_ppx005_xdpdpa_dpaetier['MOISDPAE'] = pr00_ppx005_xdpdpa_dpaetier['kd_dateembauche'].apply(lambda x:str(x.year)+str(x.month))

pr00_ppx005_xdpdpa_dpaetier['MOISDPAE2'] = pr00_ppx005_xdpdpa_dpaetier['kd_dateembauche'].apply(lambda x:str(x.year)+str(x.month+1) if x.month!=12 else str(x.year+1)+str(1))

#pr00_ppx005_xdpdpa_dpaetier = pr00_ppx005_xdpdpa_dpaetier.loc[(pr00_ppx005_xdpdpa_dpaetier['MOISDPAE']==pr00_ppx005_xdpdpa_dpaetier['MOISTAT1'])| (pr00_ppx005_xdpdpa_dpaetier['MOISDPAE2']==pr00_ppx005_xdpdpa_dpaetier['MOISTAT1'])]
pr00_ppx005_xdpdpa_dpaetier = pr00_ppx005_xdpdpa_dpaetier.query('(MOISDPAE == MOISTAT1) or (MOISDPAE2==MOISTAT1) ')

pr00_ppx005_xdpdpa_dpaetier = pr00_ppx005_xdpdpa_dpaetier.drop_duplicates(subset=['kn_periodeactivitegaec','kn_individu_national','MOISTAT1'],keep='first')

#on creer un dictionnaire par ligne et puis
#le group by 'dn_quantiteactivite', 'dc_uniteactivite_id', 'kn_periodeactivitegaec'

In [286]:
var_pour_jointure = ['dc_individu_local','dc_individu_national','kd_dateembauche','kd_datenaissance']
list(pr00_ppx005_xdpdpa_dpaetier)

['td_datmaj',
 'kc_moisstatistique',
 'kd_datecreation',
 'kc_siret',
 'dc_naf_id',
 'dc_nomraisonsociale',
 'dc_codepostal',
 'dn_employeur',
 'kc_nir',
 'kd_datenaissance',
 'dc_individu_local',
 'dc_individu_national',
 'kd_dateembauche',
 'dc_typecontrat_id',
 'dd_datefincdd',
 'dc_unite',
 'dn_individu_national',
 'dd_datedebut',
 'dd_finprevisionnelle',
 'kc_resultatsbeneficiaire_id',
 'date_fin_observation',
 'type_contrat',
 'kd_dateembauche_7avant',
 'kd_dateembauche_mois_suiv',
 'kn_periodeactivitegaec',
 'kn_individu_national',
 'MOISTAT1',
 'dn_quantiteactivite',
 'dc_uniteactivite_id',
 'MOISDPAE',
 'MOISDPAE2']

In [93]:
#pr00_ppx005_xdpdpa_dpaetier.loc[(pr00_ppx005_xdpdpa_dpaetier['MOISDPAE']==pr00_ppx005_xdpdpa_dpaetier['MOISTAT1'])| (pr00_ppx005_xdpdpa_dpaetier['MOISDPAE2']==pr00_ppx005_xdpdpa_dpaetier['MOISTAT1'])]

,td_datmaj,kc_moisstatistique,kd_datecreation,kc_siret,dc_naf_id,dc_nomraisonsociale,dc_codepostal,dn_employeur,kc_nir,kd_datenaissance,dc_individu_local,dc_individu_national,kd_dateembauche,dc_typecontrat_id,dd_datefincdd,dc_unite,dn_individu_national,dd_datedebut,dd_finprevisionnelle,kc_resultatsbeneficiaire_id,date_fin_observation,type_contrat,kd_dateembauche_7avant,kd_dateembauche_mois_suiv,kn_periodeactivitegaec,kn_individu_national,MOISTAT1,dn_quantiteactivite,dc_uniteactivite_id,MOISDPAE,MOISDPAE2
1,2019-12-22 09:51:03,201911,2019-12-10,75032334700013,0121Z,SCEA ESCOGRIFFE,49320.0,20770995,2880914118188,1988-09-12,0401110807Z,2017426501,2019-11-25,1.0,2019-11-26T00:00:00.000Z,49005.0,2017426501,2019-08-19 13:00:00,2019-12-16,VA2,2020-01-16 13:00:00,CDD,2019-11-18,2019-12-31,1696903065534471,2017426501,201912,153.0,2.0,201911,201912
8,2019-09-08 02:01:57,201909,2019-09-06,42403791900091,8610Z,UNION GESTION ETABL CAISSES ASS MALADI,76230.0,4179239,293047638425510,1993-04-29,0413626627L,8033242703,2019-09-03,1.0,2019-09-22T00:00:00.000Z,76112.0,8033242703,2019-08-29 14:30:00,2019-12-26,VEM,2020-01-26 14:30:00,CDD,2019-08-27,2019-10-31,1622686957551623,8033242703,20199,150.0,2.0,20199,201910
12,2019-10-06 03:03:39,201910,2019-10-02,83851924700020,5510Z,ACADINE PONT AUDEMER,27500.0,35067532,273102746703146,1973-10-15,0410615366B,1073264344,2019-10-01,1.0,2019-12-31T00:00:00.000Z,27056.0,1073264344,2019-08-21 14:30:00,2019-12-18,VED,2020-01-18 14:30:00,CDD,2019-09-24,2019-11-30,1642117773590535,1073264344,201910,125.0,2.0,201910,201911
13,2019-10-06 03:03:39,201910,2019-10-02,83851924700020,5510Z,ACADINE PONT AUDEMER,27500.0,35067532,273102746703146,1973-10-15,0410615366B,1073264344,2019-10-01,1.0,2019-12-31T00:00:00.000Z,27056.0,1073264344,2019-08-21 14:30:00,2019-12-18,VED,2020-01-18 14:30:00,CDD,2019-09-24,2019-11-30,1668194898067463,1073264344,201911,125.0,2.0,201910,201911
16,2019-10-06 03:03:39,201910,2019-10-02,45132990802090,7820Z,CRIT,27102.0,8761864,284055617801456,1984-05-06,0413787487J,2015814678,2019-10-01,2.0,nan,27033.0,2015814678,2019-08-22 15:30:00,2019-12-19,VEM,2020-01-19 15:30:00,CDI,2019-09-24,2019-11-30,1637146757824519,2015814678,201910,161.0,2.0,201910,201911
17,2019-10-06 03:03:39,201910,2019-10-04,41929397200081,8010Z,AGIR SECURITE,63100.0,3906759,299124101826225,1999-12-04,0357319408D,8148147986,2019-10-03,2.0,nan,63083.0,8148147986,2019-08-28 09:00:00,2019-12-26,VAB,2020-01-25 09:00:00,CDI,2019-09-26,2019-11-30,1646797117227015,8148147986,201910,159.0,2.0,201910,201911
22,2019-10-06 03:03:39,201910,2019-10-02,41881775500030,8810A,ASSOCIATION DOMICILE EMPLOIS FAMILIAUX,76170.0,3887009,284097638403176,1984-09-20,0413199072C,1070046776,2019-10-01,1.0,2020-09-30T00:00:00.000Z,76036.0,1070046776,2019-08-20 11:00:00,2019-12-17,VEM,2020-01-17 11:00:00,CDD,2019-09-24,2019-11-30,1646261642985479,1070046776,201910,210.0,2.0,201910,201911
23,2019-10-06 03:03:39,201910,2019-10-02,41881775500030,8810A,ASSOCIATION DOMICILE EMPLOIS FAMILIAUX,76170.0,3887009,284097638403176,1984-09-20,0413199072C,1070046776,2019-10-01,1.0,2020-09-30T00:00:00.000Z,76036.0,1070046776,2019-08-20 11:00:00,2019-12-17,VEM,2020-01-17 11:00:00,CDD,2019-09-24,2019-11-30,1673390647984135,1070046776,201911,116.0,2.0,201910,201911
26,2019-09-15 04:43:45,201909,2019-09-13,20001070000017,8411Z,CA CAUX SEINE AGGLO,76170.0,11275346,284097638403176,1984-09-20,0413199072C,1070046776,2019-09-12,1.0,2019-10-18T00:00:00.000Z,76036.0,1070046776,2019-08-20 11:00:00,2019-12-17,VEM,2020-01-17 11:00:00,CDD,2019-09-05,2019-10-31,1620681730195463,1070046776,20199,99.0,2.0,20199,201910
34,2019-12-08 10:50:27,201912,2019-12-03,77570019800010,5222Z,GRAND PORT MARITIME DU HAVRE,76067.0,7832542,272097638403776,1972-09-17,0413088282D,1073650826,2019-12-02,1.0,2020-03-02T00:00:00.000Z,76220.0,1073650826,2019-08-20 15:00:00,2019-12-17,VED,2020-01-17 15:00:00,CDD,2019-11-25,2020-01-31,1685865115172871,1073650826,201912,152.0,2.0,201912,20201


## Les contrats DSN, la liste des NIR est déterminé avant

In [31]:
pr00_ppx005_ddiadh_priseencharge.shape

(1073, 18)

### On commence par ramené les individue DSN

In [112]:
dsnpe_identitedeclareesal = dataiku.Dataset("dsnpe_identitedeclareesal")
pr00_pdsnpe_identitedeclareesal = dsnpe_identitedeclareesal.get_dataframe()

pr00_pdsnpe_identitedeclareesal['nir']=np.where(pr00_pdsnpe_identitedeclareesal['dc_nircertifiecnav'].isna(),pr00_pdsnpe_identitedeclareesal['dc_nirdeclare'],pr00_pdsnpe_identitedeclareesal['dc_nircertifiecnav'])

pr00_pdsnpe_identitedeclareesal = pr00_pdsnpe_identitedeclareesal.loc[pr00_pdsnpe_identitedeclareesal['nir'].isin(list_nir)]

pr00_pdsnpe_identitedeclareesal = pr00_pdsnpe_identitedeclareesal[['kn_idtidentitedeclaree','nir']].drop_duplicates(subset=['kn_idtidentitedeclaree','nir'],keep='last')

In [36]:
#pr00_pdsnpe_identitedeclareesal.head()
#pr00_ppx005_drpgep_commandebeneficiaire[['dn_individu_national','dd_datedebut','dd_finprevisionnelle','kc_resultatsbeneficiaire_id']]

### On raméne les contrats en rapport avec les identitedeclareesal 

In [256]:

list_idividu_dsn = list(pr00_pdsnpe_identitedeclareesal['kn_idtidentitedeclaree'])

dsnpe_declaractivitesal = dataiku.Dataset("dsnpe_declaractivitesal")
pr00_pdsnpe_declaractivitesal = dsnpe_declaractivitesal.get_dataframe()
pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.loc[pr00_pdsnpe_declaractivitesal['dn_idtidentitedeclaree'].isin(list_idividu_dsn)]

pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.merge(pr00_pdsnpe_identitedeclareesal,left_on=['dn_idtidentitedeclaree'],right_on=['kn_idtidentitedeclaree'],how='inner')

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (36,40,41,47,48,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [136]:
#pr00_pdsnpe_declaractivitesal.head(15)
#list(pr00_pdsnpe_declaractivitesal)

### Jointure de refh_individu et de pr00_pdsnpe_declaractivitesal sur les NIR pour avoir les BNI

In [257]:

NIR_BNI = pr00_ppx005_refh_individu[['kn_individu_national','nir']].drop_duplicates(subset=['kn_individu_national','nir'],keep='last')

list_nirbis = list(pr00_pdsnpe_declaractivitesal['nir']) 

NIR_BNI = NIR_BNI.loc[NIR_BNI['nir'].isin(list_nirbis)]

pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.merge(NIR_BNI,left_on=['nir'],right_on=['nir'],how='inner')

In [19]:
pr00_pdsnpe_declaractivitesal.shape

(11487, 67)

In [258]:
pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.merge(pr00_ppx005_drpgep_commandebeneficiaire[['dn_individu_national','dd_datedebut','dd_finprevisionnelle','kc_resultatsbeneficiaire_id']],left_on=['kn_individu_national'],right_on=['dn_individu_national'],how='inner')


In [259]:
from datetime import timedelta
pr00_pdsnpe_declaractivitesal['date_fin_observation'] = pr00_pdsnpe_declaractivitesal['dd_datedebut'].apply(lambda x:x+timedelta(days=5*30))

pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.loc[pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']>=pr00_pdsnpe_declaractivitesal['dd_datedebut']]
pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.loc[pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<=pr00_pdsnpe_declaractivitesal['date_fin_observation']]
#pr00_pdsnpe_declaractivitesal.head()

### le type de contrat DSN
* 01 - Contrat de travail à durée indéterminée de droit privé
* 02 - Contrat de travail à durée déterminée de droit privé
* 03 - Contrat de mission (contrat de travail temporaire)
* 07 - Contrat à durée indéterminée intermittent
* 08 - Contrat à durée indéterminée intérimaire
* 09 - Contrat de travail à durée indéterminée de droit public
* 10 - Contrat de travail à durée déterminée de droit public
* 20 - [FP] Détachement d’un agent d’une Fonction Publique donnant lieu à pension (ECP)
* 21 - [FP] Détachement d’un agent d’une Fonction Publique ne donnant pas lieu à pension (ENCP)
* 29 - Convention de stage (hors formation professionnelle)
* 32 - Contrat d’appui au projet d’entreprise
* 50 - Nomination dans la fonction publique (par arrêté, par décision,…)
* 60 - Contrat d'engagement éducatif
* 70 - Contrat de soutien et d'aide par le travail
* 80 - Mandat social
* 81 - Mandat d'élu
* 82 - Contrat de travail à durée indéterminée de Chantier ou d'opération
* 89 - Volontariat de service civique
* 90 - Autre nature de contrat, convention, mandat

In [260]:
import numpy as np
conditions  = [pr00_pdsnpe_declaractivitesal['dc_naturecontrat_id'].isin([1,7,8,9,82]),\
              pr00_pdsnpe_declaractivitesal['dc_naturecontrat_id'].isin([2,10]),\
              pr00_pdsnpe_declaractivitesal['dc_naturecontrat_id'].isin([3])]

values = ['CDI','CDD','CTT']
pr00_pdsnpe_declaractivitesal['type_contrat'] = np.select(conditions, values, 'NR')

### les dates de fin de contrat

In [261]:
pr00_pdsnpe_declaractivitesal['dd_datefincontrat']= pr00_pdsnpe_declaractivitesal[['dd_datefinprevcontrat','dd_datefincontrat']].max(axis=1)
#on enlève les ligne avec NaT
#pr00_pdsnpe_declaractivitesal['dd_datefincontrat'].loc[pr00_pdsnpe_declaractivitesal['dd_datefincontrat'].isna()]

### Les quantités de travail de la DSN: On convertie les quantité de travail en heure
proportion sur le nombre de jour par mois (5jours par semaine, on multiplie par 21

In [262]:
#dc_modalitetpstravail_id = 10
#dc_unitequotitetravail_id',
 #dn_quotitetravail',
 #dc_modalitetpstravail_id',10 - heure
unitequotitetravail = {12 :'journée',20 :'forfait jour',21: 'forfait heure',31:'à la pige',32: 'à la vacation',\
                       33 :'à la tâche',90 :'salarié sans unité de mesure définie au contrat',99 :'salarié non concerné',\
                       10:'heure'}


modalitetpstravail = {10:'temps plein',20 :'temps partiel',21:'temps partiel thérapeutique',99:'salarié non concerné'}

hh = pr00_pdsnpe_declaractivitesal.groupby(['dc_unitequotitetravail_id','dn_quotitetravail']).agg({'dn_idtidentitedeclaree':'count'}).reset_index().sort_values(by='dn_idtidentitedeclaree',ascending=False)
hh['unite'] = hh['dc_unitequotitetravail_id'].apply(lambda x:unitequotitetravail[x])

#10 et 21 heure on ne change rien, 
#12 et 20 journe : nbheure = nbjour*6.93

import numpy as np
conditions = [pr00_pdsnpe_declaractivitesal['dc_unitequotitetravail_id'].isin([10,21,'10','21']),\
             pr00_pdsnpe_declaractivitesal['dc_unitequotitetravail_id'].isin([12,20,'12','20']),\
             pr00_pdsnpe_declaractivitesal['dc_unitequotitetravail_id'].isin([99,90])]

values = [pr00_pdsnpe_declaractivitesal['dn_quotitetravail'],pr00_pdsnpe_declaractivitesal['dn_quotitetravail']*6.94,pr00_pdsnpe_declaractivitesal['dn_quotitetravail']]

#quotiteheure = nombre d'heure dans le mois
pr00_pdsnpe_declaractivitesal['quotiteheure'] = np.select(conditions, values, pr00_pdsnpe_declaractivitesal['dn_quotitetravail'])

#conditionbis = [(pr00_pdsnpe_declaractivitesal['dc_modalitetpstravail_id'].isin([10,'10']))|pr00_pdsnpe_declaractivitesal['quotiteheure']>77.5]
                
pr00_pdsnpe_declaractivitesal['sup_78_heure'] = np.where((pr00_pdsnpe_declaractivitesal['dc_modalitetpstravail_id'].isin([10,'10']))|(pr00_pdsnpe_declaractivitesal['quotiteheure']>77.5),'sup_78','inf_78')

### Calcule de nouvelle date de fin et de débute: cas de l'activité réduite
Pour Les CDD
* Si la date de début de contrat est comprise dans le mois principale on laisse cette date.
* Si la date de début de contrat est inférieure 

https://docs.python.org/3/library/datetime.html

In [273]:
#dd_moisprincipaldeclare, dd_datedebutcontrat, dd_datefincontrat, 
#type_contrat , quotiteheure

import calendar
from datetime import datetime, date


new_date = pd.to_datetime(pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat'].isin(['CTT','CDD'])) & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])].copy()['dd_datedebutcontrat'].dt.date)            
pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat'].isin(['CTT','CDD'])) & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']),'dd_moisprincipaldeclare'] =  new_date

pr00_pdsnpe_declaractivitesal['fin_moisprincipale'] = pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'].apply(lambda x:datetime(x.year,x.month,calendar.monthrange(x.year,x.month)[1]))
pr00_pdsnpe_declaractivitesal['date_debut_travail_mois'] = pr00_pdsnpe_declaractivitesal[['dd_moisprincipaldeclare','dd_datedebutcontrat']].max(axis=1).apply(lambda x:datetime(x.year,x.month,x.day))
pr00_pdsnpe_declaractivitesal['date_fin_travail_mois'] = pr00_pdsnpe_declaractivitesal[['fin_moisprincipale','dd_datefincontrat']].min(axis=1).apply(lambda x:datetime(x.year,x.month,x.day))

#hh = pr00_pdsnpe_declaractivitesal.loc[pr00_pdsnpe_declaractivitesal['dc_modalitetpstravail_id'].isin([20,21,'20','21'])]
#hh[['dd_datedebutcontrat','dd_moisprincipaldeclare','fin_moisprincipale','date_debut_travail_mois','date_fin_travail_mois','dd_datefincontrat']].tail()

In [275]:
def calcul_nb_jour(x,y):
    list_jour = [o.isocalendar() for o in pd.date_range(start=x, end=y, freq='D')]
    list_semaine = list(set([o[1] for o in list_jour]))
    if len(list_jour )>=7:
        list_jour = [o for o in list_jour if o[2] not in [6,7]]
    list_jour_semaine = [([o[2] for o in list_jour if o[1]==sem],sem) for sem in list_semaine] 
    nb_jour_sem = [[len(o[0]) if len(o[0])<=5 else len(o[0]),o[1]] for o in list_jour_semaine]
    nb_jour = sum([o[0] for o in nb_jour_sem])
    return(nb_jour)
    
def calcul_nb_jourbis(x,y):
    list_jour = [o.isocalendar() for o in pd.date_range(start=x, end=y, freq='D')]
    nbjour = len([o for o in list_jour if o[2] not in [6,7]])
    return(nbjour)

pr00_pdsnpe_declaractivitesal['nb_jour_trav_mois'] = pr00_pdsnpe_declaractivitesal[['date_debut_travail_mois','date_fin_travail_mois']].apply(lambda x:calcul_nb_jour(x[0],x[1]),axis=1)

In [282]:
#dd_moisprincipaldeclare
#pr00_pdsnpe_declaractivitesal['type_contrat']
import numpy as np
conditions = [(pr00_pdsnpe_declaractivitesal['dc_modalitetpstravail_id'].isin([20,21,'20','21'])) \
              & (pr00_pdsnpe_declaractivitesal['type_contrat'].isin(['CTT','CDD','CDI'])) & \
              (pr00_pdsnpe_declaractivitesal['quotiteheure']<78) & \
              (pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']<=pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']),\
                 (pr00_pdsnpe_declaractivitesal['dc_modalitetpstravail_id'].isin([20,21,'20','21'])) \
              & (pr00_pdsnpe_declaractivitesal['type_contrat'].isin(['CTT','CDD','CDI'])) & \
              (pr00_pdsnpe_declaractivitesal['quotiteheure']<78) & \
              (pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']>=pr00_pdsnpe_declaractivitesal['dd_datefincontrat'])]
             

values = [(pr00_pdsnpe_declaractivitesal['quotiteheure']*21)/pr00_pdsnpe_declaractivitesal['nb_jour_trav_mois'] \
          ,(pr00_pdsnpe_declaractivitesal['quotiteheure']*21)/pr00_pdsnpe_declaractivitesal['nb_jour_trav_mois']]

#quotiteheure = nombre d'heure dans le mois
pr00_pdsnpe_declaractivitesal['quotiteheurebis'] = np.select(conditions, values, pr00_pdsnpe_declaractivitesal['quotiteheure'])


In [284]:
### Brouillon
pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & (pr00_pdsnpe_declaractivitesal['dc_modalitetpstravail_id'].isin([20,21,'20','21']))]

,kd_dateextraction,kn_idtdeclarationactivite,dn_idtidentitedeclaree,dc_idt1declaration,dc_idt2declaration,dc_idtmetier,dc_naturedeclaration_id,dn_typecalculdeclaration_id,dc_typedeclaration_id,dd_moisprincipaldeclare,dc_champdeclaration_id,dc_indicateurvalidite_id,dn_effectifdeclare,dc_numerocontrat,dc_naturecontrat_id,dd_datedebutcontrat,dd_datefinprevcontrat,dd_datefincontrat,dc_motifrupturecontrat_id,dc_typeembauche_id,dc_statutsalarie_id,dc_profcatsoc_id,dc_complprofcatsoc_id,dc_emploi,dc_dispositifpolitique_id,dc_unitequotitetravail_id,dn_quotitetravail,dc_modalitetpstravail_id,dc_motifrecours_id,dc_typetravetranger_id,dc_regimeretraitecompl_id,dc_statutemploisalarie_id,dc_typegestionac_id,dd_dateadhesion,dd_datedenonciation,dd_dateeffetconventiongestion,dc_numeroconventiongestion,dc_heureembauchedpae,dc_typeemploimultiple_id,dc_typeemployeurmultiple_id,dc_numerolabel,dc_numerolicence,dc_siretetbaffectation,dc_siretetbutilisateur,dc_idtetbutilisateur,dc_codepostaletbutilisateur,dc_localiteetbutilisateur,dc_pays_etbutilisateur_id,dc_distribetrangeretbutil,dc_communeinseeetbutilisateur,dc_siretetblieutravail,dc_idtetblieutravail,dc_codepostaletblieutravail,dc_localiteetblieutravail,dc_pays_etblieutravail_id,dc_distribetrangeretb,dc_communeinseeetblieutravail,dd_datedernierjourtravailpaye,dc_modfincontrat_id,filename,date_import_raw,date_insertion_clean,yearraff,monthraff,kn_idtidentitedeclaree,nir,kn_individu_national,dn_individu_national,dd_datedebut,dd_finprevisionnelle,kc_resultatsbeneficiaire_id,date_fin_observation,type_contrat,quotiteheure,sup_78_heure,fin_moisprincipale,date_debut_travail_mois,date_fin_travail_mois,nb_jour_trav_mois,quotiteheurebis
70,2019-10-24 18:00:00,477632742,95901002,WStt3vWBFA9JZ6Rsi9CiLtN,1,194001025,1,2,1,2019-10-01,1,N,8.0,00003401102019,2,2019-10-01,2020-04-04,2020-04-04,NaN,NaN,6,554e,NaN,HOTESSE DE VENTES,99,10,104.00,20,2.0,99,RUAA,99,NaN,NaT,NaT,NaT,NaN,NaN,3,1,NaN,NaN,82112890700028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,remise.j.DSNPESPM.edi.DSNDECSAL.S3648922.DAT_2...,2019-10-24 21:04:22,2019-10-25 02:00:55.340,2019,10,95901002,2971047001175,8150474431,8150474431,2019-08-30 09:30:00,2019-12-27,VEM,2020-01-27 09:30:00,CDD,104.00,sup_78,2019-10-31,2019-10-01,2019-10-31,23,104.000000
71,2019-11-29 18:00:00,490410054,95901002,WS8DxvWBFAz4o6RkfEOlEeQ,1,195118615,1,2,1,2019-10-01,1,NaN,9.0,00003401102019,2,2019-10-01,2020-04-04,2020-04-04,NaN,NaN,6,554e,NaN,HOTESSE DE VENTES,99,10,107.60,20,2.0,99,RUAA,99,NaN,NaT,NaT,NaT,NaN,NaN,3,1,NaN,NaN,82112890700028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,remise.j.DSNPESPM.edi.DSNDECSAL.S2424603.DAT_2...,2019-11-29 21:09:03,2019-11-30 02:01:01.002,2019,11,95901002,2971047001175,8150474431,8150474431,2019-08-30 09:30:00,2019-12-27,VEM,2020-01-27 09:30:00,CDD,107.60,sup_78,2019-10-31,2019-10-01,2019-10-31,23,107.600000
72,2019-12-16 18:00:00,477632742,95901002,WStt3vWBFA9JZ6Rsi9CiLtN,1,194001025,1,2,1,2019-10-01,1,N,8.0,00003401102019,2,2019-10-01,2020-04-04,2020-04-04,NaN,NaN,6,554e,NaN,HOTESSE DE VENTES,99,10,104.00,20,2.0,99,RUAA,99,NaN,NaT,NaT,NaT,NaN,NaN,3,1,NaN,NaN,82112890700028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,remise.j.DSNPESPM.edi.DSNDECSAL.S2187549.DAT_1...,2019-12-16 23:08:51,2019-12-17 02:01:00.806,2019,12,95901002,2971047001175,8150474431,8150474431,2019-08-30 09:30:00,2019-12-27,VEM,2020-01-27 09:30:00,CDD,104.00,sup_78,2019-10-31,2019-10-01,2019-10-31,23,104.000000
73,2019-12-16 18:00:00,490410054,95901002,WS8DxvWBFAz4o6RkfEOlEeQ,1,195118615,1,2,1,2019-10-01,1,NaN,9.0,00003401102019,2,2019-10-01,2020-04-04,2020-04-04,NaN,NaN,6,554e,NaN,HOTESSE DE VENTES,99,10,107.60,20,2.0,99,RUAA,99,NaN,NaT,NaT,NaT,NaN,NaN,3,1,NaN,NaN,82112890700028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,remise.j.DSNPESPM.edi.DSNDECSAL.S2187549.DAT_1...,2019-12-16 23:08:51,2019-12-17 02:01:00.806,2019,12,95901002,2971047001175,8150474431,8150474431,2019-08-30 09:30:

In [255]:
import calendar
from datetime import datetime, date
import numpy as np

bug = pr00_pdsnpe_declaractivitesal.query("type_contrat=='CDD' and dd_datedebutcontrat<dd_moisprincipaldeclare and dd_datefincontrat<dd_moisprincipaldeclare")
#bug
#np.where((pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & \
         #(pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']) &\
        #(pr00_pdsnpe_declaractivitesal['dd_datefincontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']),\
         #pd.to_datetime(pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat'].dt.year,pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat'].dt.month,1),pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])

#pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])]['dd_moisprincipaldeclare']
from datetime import datetime, date
import numpy as np

bug = pr00_pdsnpe_declaractivitesal.query("type_contrat=='CDD' and dd_datedebutcontrat<dd_moisprincipaldeclare and dd_datefincontrat<dd_moisprincipaldeclare")
#bug
#np.where((pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & \
         #(pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']) &\
        #(pr00_pdsnpe_declaractivitesal['dd_datefincontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare']),\
         #pd.to_datetime(pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat'].dt.year,pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat'].dt.month,1),pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])

new_date = pd.to_datetime(pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])].copy()['dd_datedebutcontrat'].dt.date)            
pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])]['dd_moisprincipaldeclare'] =  new_date

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [253]:
#pd.to_datetime(pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])].copy()['dd_datedebutcontrat'].dt.date)
pr00_pdsnpe_declaractivitesal.loc[(pr00_pdsnpe_declaractivitesal['type_contrat']=='CDD') & (pr00_pdsnpe_declaractivitesal['dd_datedebutcontrat']<pr00_pdsnpe_declaractivitesal['dd_moisprincipaldeclare'])]

0       2019-09-04
3       2019-09-04
4       2019-09-04
5       2019-09-04
7       2019-09-04
8       2019-09-04
9       2019-09-04
17      2019-10-01
18      2019-10-01
19      2019-10-01
38      2019-09-09
40      2019-09-09
41      2019-09-09
42      2019-09-09
43      2019-09-09
45      2019-09-09
56      2019-10-01
57      2019-10-01
59      2019-10-01
60      2019-10-01
71      2019-10-01
73      2019-10-01
74      2019-10-01
75      2019-10-01
76      2019-09-19
79      2019-09-19
82      2019-09-30
83      2019-09-30
84      2019-09-30
85      2019-09-30
86      2019-09-30
87      2019-09-30
88      2019-09-30
89      2019-09-30
90      2019-09-30
91      2019-09-30
92      2019-09-30
93      2019-09-30
94      2019-09-30
95      2019-09-30
96      2019-09-30
97      2019-09-30
98      2019-09-30
99      2019-09-30
100     2019-09-30
101     2019-09-30
121     2019-10-14
122     2019-10-14
124     2019-10-14
133     2019-10-14
134     2019-10-14
136     2019-10-14
137     2019

In [151]:
list(pr00_pdsnpe_declaractivitesal)

['kd_dateextraction',
 'kn_idtdeclarationactivite',
 'dn_idtidentitedeclaree',
 'dc_idt1declaration',
 'dc_idt2declaration',
 'dc_idtmetier',
 'dc_naturedeclaration_id',
 'dn_typecalculdeclaration_id',
 'dc_typedeclaration_id',
 'dd_moisprincipaldeclare',
 'dc_champdeclaration_id',
 'dc_indicateurvalidite_id',
 'dn_effectifdeclare',
 'dc_numerocontrat',
 'dc_naturecontrat_id',
 'dd_datedebutcontrat',
 'dd_datefinprevcontrat',
 'dd_datefincontrat',
 'dc_motifrupturecontrat_id',
 'dc_typeembauche_id',
 'dc_statutsalarie_id',
 'dc_profcatsoc_id',
 'dc_complprofcatsoc_id',
 'dc_emploi',
 'dc_dispositifpolitique_id',
 'dc_unitequotitetravail_id',
 'dn_quotitetravail',
 'dc_modalitetpstravail_id',
 'dc_motifrecours_id',
 'dc_typetravetranger_id',
 'dc_regimeretraitecompl_id',
 'dc_statutemploisalarie_id',
 'dc_typegestionac_id',
 'dd_dateadhesion',
 'dd_datedenonciation',
 'dd_dateeffetconventiongestion',
 'dc_numeroconventiongestion',
 'dc_heureembauchedpae',
 'dc_typeemploimultiple_id',
 '

## Les lignes GAEC
* dc_typeperiodegaec_id : ('9' Contrat de travail), ('7' Bulletin de salaire), ('16' Chèque emploi service)
* 

In [119]:
drvder_periodeactivitegaec = dataiku.Dataset("drvder_periodeactivitegaec")
pr00_ppx005_drvder_periodeactivitegaec = drvder_periodeactivitegaec.get_dataframe()

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dc_typeperiodegaec_id'].isin(['9','7','16'])]

#on enleve les origine 3
list_origineinformation = ['15','86','37','2','12','4','38','54','33','110','34','56','3','5','55','7','1']

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dc_origineinformationgaec_id'].isin(list_origineinformation)]


#A suprimer
pr00_ppx005_drvder_periodeactivitegaec['dc_motiffct_id'] = pr00_ppx005_drvder_periodeactivitegaec['dc_motiffct_id'].fillna(2000)
pr00_ppx005_drvder_periodeactivitegaec['dc_motiffct_id'] = pr00_ppx005_drvder_periodeactivitegaec['dc_motiffct_id'].apply(lambda x:str(int(x)))


pr00_ppx005_ref_motiffct = dataiku.Dataset("pr00_ppx005_ref_motiffct")
pr00_ppx005_ref_motiffct = pr00_ppx005_ref_motiffct.get_dataframe()
pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.merge(pr00_ppx005_ref_motiffct[['kc_motiffct','dc_lblmotiffct']],how='inner',left_on=['dc_motiffct_id'],right_on=['kc_motiffct'])

#
list_BNI = list(pr00_ppx005_drpgep_commandebeneficiaire['dn_individu_national'])

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['kn_individu_national'].isin(list_BNI)]

### Jointure avec origine information GAEC : pr00_ppx005_ref_origineinformationgaec

In [120]:
pr00_ppx005_ref_origineinformationgaec = dataiku.Dataset("pr00_ppx005_ref_origineinformationgaec")
pr00_ppx005_ref_origineinformationgaec = pr00_ppx005_ref_origineinformationgaec.get_dataframe()

ref_origineinformationgaec = pr00_ppx005_ref_origineinformationgaec[['kc_origineinformationgaec','dc_lblorigineinformationgaec']]

pr00_ppx005_drvder_periodeactivitegaec['dc_origineinformationgaec_id'] = pr00_ppx005_drvder_periodeactivitegaec['dc_origineinformationgaec_id'].apply(lambda x:str(int(x)))

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.merge(ref_origineinformationgaec,how='inner',left_on=['dc_origineinformationgaec_id'],right_on=['kc_origineinformationgaec'])

### Jointure avec commande bénificaire et filtre sur les date 

In [121]:
#pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.merge()
drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire[['dn_individu_national','dd_datedebut','dd_finprevisionnelle','kc_resultatsbeneficiaire_id']]
pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.merge(drpgep_commandebeneficiaire,left_on=['kn_individu_national'],\
                                                                                      right_on=['dn_individu_national'],how='inner')

#On calcule la date de fin d'observation
from datetime import timedelta
pr00_ppx005_drvder_periodeactivitegaec['date_fin_observation'] = pr00_ppx005_drvder_periodeactivitegaec['dd_datedebut'].apply(lambda x:x+timedelta(days=5*30))

pr00_ppx005_drvder_periodeactivitegaec['dd_datefinactivite'] = pd.to_datetime(pr00_ppx005_drvder_periodeactivitegaec['dd_datefinactivite'], format="%Y-%m-%dT%H:%M:%S.%fZ", errors='ignore')

In [122]:
pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dd_datedebutactivite']>=pr00_ppx005_drvder_periodeactivitegaec['dd_datedebut']]

pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.loc[pr00_ppx005_drvder_periodeactivitegaec['dd_datedebutactivite']<=pr00_ppx005_drvder_periodeactivitegaec['date_fin_observation']]

### On détermine le type de contrat (ne plus faire on veut une date debut et fin activité)

In [123]:
import numpy as np
conditions = [pr00_ppx005_drvder_periodeactivitegaec['kc_motiffct'].isin(['12','15','16','7','46']),\
             pr00_ppx005_drvder_periodeactivitegaec['kc_motiffct'].isin(['8']),\
             pr00_ppx005_drvder_periodeactivitegaec['dd_datefinactivite'].isna()]

values = ['CDD','CTT','CDI']
pr00_ppx005_drvder_periodeactivitegaec['type_contrat'] = np.select(conditions, values, 'NR')

### Les tables intérims

In [ ]:
#beneficiaireeaem_df = ... # Compute a Pandas dataframe to write into BENEFICIAIREEAEM
# Write recipe outputs
#beneficiaireeaem = dataiku.Dataset("BENEFICIAIREEAEM")
#beneficiaireeaem.write_with_schema(beneficiaireeaem_df)


In [24]:
pr00_ppx005_drvder_periodeactivitegaec.shape

(869, 21)

In [92]:
#pr00_ppx005_drvder_periodeactivitegaec['dd_datefinactivite'].apply(lambda x:datetime.strptime(str(x), "%Y-%m-%dT%H:%M:%S.%fZ"))
#pd.to_datetime(pr00_ppx005_drvder_periodeactivitegaec['dd_datefinactivite'], format="%Y-%m-%dT%H:%M:%S.%fZ", errors='ignore')


### Pour agragation chiffrage pour Nicolas

In [30]:
#pr00_ppx005_drpgep_commandebeneficiaire.head()
#pr00_ppx005_xdpdpa_dpae
#pr00_ppx005_drvder_periodeactivitegaec
#pr00_pdsnpe_declaractivitesal
#agregadsn = pr00_pdsnpe_declaractivitesal.

pr00_pdsnpe_declaractivitesal = pr00_pdsnpe_declaractivitesal.drop_duplicates(subset=['dn_idtidentitedeclaree','dc_numerocontrat'],keep='last')
pr00_ppx005_xdpdpa_dpae = pr00_ppx005_xdpdpa_dpae.drop_duplicates(subset=['dn_employeur','dc_individu_national','kd_dateembauche'],keep='last')
pr00_ppx005_drvder_periodeactivitegaec = pr00_ppx005_drvder_periodeactivitegaec.drop_duplicates(subset=['kn_individu_national','kn_periodeactivitegaec' ,'dd_datedebutactivite','dn_entreprise_id'],keep='last')

pr00_ppx005_drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire.drop_duplicates(subset=['dc_codesiteregion_id','dc_typepresta_id_x','dc_commandepresta','dn_individu_national','dd_datedebut'],keep='last')

In [52]:
#dc_codesiteregion_id, dc_typepresta_id_x, dc_commandepresta,dn_individu_national,dd_datedebut
#pr00_ppx005_drpgep_commandebeneficiaire = pr00_ppx005_drpgep_commandebeneficiaire.drop_duplicates(subset=['dc_codesiteregion_id','dc_typepresta_id_x','dc_commandepresta','dn_individu_national','dd_datedebut'],keep='last')

In [31]:
# kc_siret, dc_individu_national, kd_dateembauche ,dc_typecontrat_id, dn_employeur

NB_DSN = pr00_pdsnpe_declaractivitesal.groupby(['kn_individu_national']).agg({'kn_idtdeclarationactivite':'count'}).reset_index().rename(columns={'kn_idtdeclarationactivite':'nb_dsn'})

NB_DPAE = pr00_ppx005_xdpdpa_dpae.groupby(['dc_individu_national']).agg({'dn_employeur':'count'}).reset_index().rename(columns={'dn_employeur':'nb_dpae'})

NB_GAEC = pr00_ppx005_drvder_periodeactivitegaec.groupby(['kn_individu_national']).agg({'kn_periodeactivitegaec':'count'}).reset_index().rename(columns={'kn_periodeactivitegaec':'nb_gaec'})

In [32]:
#pr00_ppx005_drvder_periodeactivitegaec
#pr00_ppx005_drpgep_commandebeneficiairebis = pr00_ppx005_drpgep_commandebeneficiaire.merge(NB_DSN,how='inner',left_on=['kn_individu_national'],right_on=)
#NBDECLA = NB_DSN.merge(NB_DPAE,how='outer',left_on='kn_individu_national',right_on='dc_individu_national')
#NBDECLA = NBDECLA.merge(NB_GAEC,how='outer,left_on=')
pr00_ppx005_drpgep_commandebeneficiairebis = pr00_ppx005_drpgep_commandebeneficiaire.merge(NB_DSN,how='left',left_on=['dn_individu_national'],right_on='kn_individu_national')
pr00_ppx005_drpgep_commandebeneficiaire3 = pr00_ppx005_drpgep_commandebeneficiairebis.merge(NB_DPAE,how='left',left_on=['dn_individu_national'],right_on='dc_individu_national')
pr00_ppx005_drpgep_commandebeneficiaire4 = pr00_ppx005_drpgep_commandebeneficiaire3.merge(NB_GAEC,how='left',left_on=['dn_individu_national'],right_on='kn_individu_national')

In [33]:
resultat = {'VA2':"SORTIE ANTICIPEE/ABANDON A PARTIR DU 2EME MOIS",\
            'VAB':"SORTIE ANTICIPEE/ABANDON AVANT LE 2EME MOIS",\
            'VAT':"A SUIVI LA PRESTATION DANS SA TOTALITE",\
            'VED':"REPRISE D'EMPLOI CDD OU MISSION INTERIM DE 3 A 6 MOIS",\
            'VEM':"REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM >= A 6 MOIS"}




pr00_ppx005_drpgep_commandebeneficiaire4['resultat_libele'] = pr00_ppx005_drpgep_commandebeneficiaire4['kc_resultatsbeneficiaire_id'].apply(lambda x:resultat[x])

In [34]:
#contrat_dpae = {1.0:'CDD',2.0:'CDI',3.0:'CTT'}
#import numpy as np
#conditions = [pr00_ppx005_xdpdpa_dpae['dc_typecontrat_id']==1.0,\
             #pr00_ppx005_xdpdpa_dpae['dc_typecontrat_id']==2.0,\
             #pr00_ppx005_xdpdpa_dpae['dc_typecontrat_id']==3.0]

#values = ['CDD','CDI','CTT']
#pr00_ppx005_xdpdpa_dpae['type_contrat'] = np.select(conditions, values, 'NR')
pr00_ppx005_drpgep_commandebeneficiaire4

,dc_codesiteregion_id,dc_typepresta_id_x,dc_commandepresta,kn_idcommandepresta_x,kn_commandebeneficiaire,dn_individu_national,dc_individu_local,dd_datedebut,dd_finprevisionnelle,dc_typepresta_id_y,kn_idcommandepresta_y,kn_commandebeneficiaire_id,kc_resultatsbeneficiaire_id,dd_dateeffetresultat,resultat_libele,kn_individu_national_x,nb_dsn,dc_individu_national,nb_dpae,kn_individu_national_y,nb_gaec
0,13992,AEM,JB011,7500160,25570701,8149421205,0348582301N,2019-08-13 10:00:00,2019-12-10,AEM,7500160,25570701,VAT,NaT,A SUIVI LA PRESTATION DANS SA TOTALITE,NaN,NaN,NaN,NaN,NaN,NaN
1,13992,AEM,JB022,7500171,25668474,2054176605,0325333870K,2019-08-13 15:00:00,2019-12-10,AEM,7500171,25668474,VA2,2019-10-14,SORTIE ANTICIPEE/ABANDON A PARTIR DU 2EME MOIS,NaN,NaN,NaN,NaN,NaN,NaN
2,13992,AEM,JB036,7500185,25619327,2088496531,0347905362K,2019-08-13 14:00:00,2019-12-10,AEM,7500185,25619327,VEM,2019-10-02,REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM ...,2.088497e+09,2.0,2.088497e+09,1.0,NaN,NaN
3,13992,AEM,JB050,7502292,25499313,2053751017,0348352884X,2019-08-14 09:30:00,2019-12-11,AEM,7502292,25499313,VAB,2019-08-15,SORTIE ANTICIPEE/ABANDON AVANT LE 2EME MOIS,2.053751e+09,1.0,2.053751e+09,2.0,NaN,NaN
4,13992,AEM,JB078,7506227,25731196,1192185596,0347118028M,2019-08-19 09:30:00,2019-12-16,AEM,7506227,25731196,VAB,2019-08-20,SORTIE ANTICIPEE/ABANDON AVANT LE 2EME MOIS,1.192186e+09,4.0,1.192186e+09,4.0,NaN,NaN
5,13992,AEM,JB157,7509950,25590134,1197414359,0346943604W,2019-08-20 14:00:00,2019-12-17,AEM,7509950,25590134,VED,2019-11-18,REPRISE D'EMPLOI CDD OU MISSION INTERIM DE 3 A...,1.197414e+09,4.0,1.197414e+09,4.0,NaN,NaN
6,13992,AEM,JB190,7516766,25728572,2012042273,0347384420U,2019-08-22 14:00:00,2019-12-19,AEM,7516766,25728572,VED,2019-09-16,REPRISE D'EMPLOI CDD OU MISSION INTERIM DE 3 A...,2.012042e+09,1.0,2.012042e+09,1.0,NaN,NaN
7,13992,AEM,JB207,7516783,25636134,1952766179,0348134509H,2019-08-22 14:00:00,2019-12-19,AEM,7516783,25636134,VED,2019-10-01,REPRISE D'EMPLOI CDD OU MISSION INTERIM DE 3 A...,1.952766e+09,1.0,1.952766e+09,1.0,NaN,NaN
8,13992,AEM,JB213,7516789,25560356,1190478555,0342505079M,2019-08-22 14:00:00,2019-12-19,AEM,7516789,25560356,VAB,2019-08-23,SORTIE ANTICIPEE/ABANDON AVANT LE 2EME MOIS,1.190479e+09,2.0,NaN,NaN,1.190479e+09,1.0
9,13992,AEM,JB219,7516795,25730468,1196746076,0346982474W,2019-08-22 09:30:00,2019-12-19,AEM,7516795,25730468,VAT,NaT,A SUIVI LA PRESTATION DANS SA TOTALITE,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
#Contrat.StatutConventionnel
#Le statut détermine les conditions
#ref_resultatbeneficiaire = dataiku.Dataset("ref_resultatbeneficiaire")
#pr00_ppx005_ref_resultatbeneficiaire = ref_resultatbeneficiaire.get_dataframe()
#pr00_pdsnpe_declaractivitesal.head()

#Attention fait avant
import numpy as np
conditions  = [pr00_pdsnpe_declaractivitesal['dc_naturecontrat_id'].isin([1,7,8,9,82]),\
              pr00_pdsnpe_declaractivitesal['dc_naturecontrat_id'].isin([2,10]),\
              pr00_pdsnpe_declaractivitesal['dc_naturecontrat_id'].isin([3])]

values = ['CDI','CDD','CTT']
pr00_pdsnpe_declaractivitesal['type_contrat'] = np.select(conditions, values, 'NR')

In [125]:
###Avant l'union des tables
columns_to_keep_dsn =[ 'kn_idtdeclarationactivite' ,'dn_idtidentitedeclaree'\
                      ,'dc_numerocontrat','dd_datedebutcontrat','dd_datefincontrat',\
                     'dc_unitequotitetravail_id','dn_quotitetravail','dc_modalitetpstravail_id',\
                      'dc_siretetbaffectation','nir','kn_individu_national','type_contrat',\
                     'dd_datedebut','dd_finprevisionnelle','kc_resultatsbeneficiaire_id', 'date_fin_observation']

pr00_pdsnpe_declaractivitesal_keep = pr00_pdsnpe_declaractivitesal[columns_to_keep_dsn]

In [126]:
pr00_pdsnpe_declaractivitesal_keep['idendifiant'] = pr00_pdsnpe_declaractivitesal_keep[['kn_idtdeclarationactivite' ,\
                                                                                        'dn_idtidentitedeclaree','dc_numerocontrat']].apply(lambda x:{'kn_idtdeclarationactivite':x[0],'dn_idtidentitedeclaree':x[1],'dc_numerocontrat':x[2]},axis=1)

pr00_pdsnpe_declaractivitesal_keep['quantite_travaille'] = pr00_pdsnpe_declaractivitesal_keep[['dc_unitequotitetravail_id','dn_quotitetravail','dc_modalitetpstravail_id']].apply(lambda x:{'dc_unitequotitetravail_id':x[0],'dn_quotitetravail':x[1],'dc_modalitetpstravail_id':x[2]},axis=1)

pr00_pdsnpe_declaractivitesal_keep['source'] = 'DSN'
pr00_pdsnpe_declaractivitesal_keep['BNE'] = np.nan

columns_to_keep = ['source','dc_siretetbaffectation','BNE','nir','kn_individu_national','type_contrat','dd_datedebutcontrat',\
                   'dd_datefincontrat','idendifiant','quantite_travaille','dd_datedebut',\
                   'kc_resultatsbeneficiaire_id', 'date_fin_observation']

pr00_pdsnpe_declaractivitesal_keep = pr00_pdsnpe_declaractivitesal_keep[columns_to_keep]


columns_to_rename = {'dc_siretetbaffectation':'siret','BNE':'BNE','nir':'nir','kn_individu_national':'BNI',\
                    'dd_datedebutcontrat':'datedebucontrat','dd_datefincontrat':'datefincontrat'}

pr00_pdsnpe_declaractivitesal_keep = pr00_pdsnpe_declaractivitesal_keep.rename(columns=columns_to_rename)

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipy

### DPAE

In [127]:
colomns_to_keep = ['kc_siret','dn_employeur','kc_nir','dc_individu_national',\
                   'kd_dateembauche','dd_datefincdd','type_contrat',\
                  'dd_datedebut','kc_resultatsbeneficiaire_id','date_fin_observation']

pr00_ppx005_xdpdpa_dpae_keep = pr00_ppx005_xdpdpa_dpae[colomns_to_keep]
pr00_ppx005_xdpdpa_dpae_keep['source'] = 'DPAE'
pr00_ppx005_xdpdpa_dpae_keep['idendifiant'] = ' '
pr00_ppx005_xdpdpa_dpae_keep['quantite_travaille'] = ' '



colomns_to_keep = ['source','kc_siret','dn_employeur','kc_nir','dc_individu_national',\
                   'type_contrat', 'kd_dateembauche','dd_datefincdd','idendifiant',\
                   'quantite_travaille','dd_datedebut',\
                   'kc_resultatsbeneficiaire_id', 'date_fin_observation']

pr00_ppx005_xdpdpa_dpae_keep = pr00_ppx005_xdpdpa_dpae_keep[colomns_to_keep]


columns_to_rename = {'kc_siret':'siret','dn_employeur':'BNE','kc_nir':'nir','dc_individu_national':'BNI',\
                    'kd_dateembauche':'datedebucontrat','dd_datefincdd':'datefincontrat'}

pr00_ppx005_xdpdpa_dpae_keep = pr00_ppx005_xdpdpa_dpae_keep.rename(columns=columns_to_rename)

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [128]:
colomns_to_keep = ['kn_periodeactivitegaec','dc_origineinformationgaec_id','dc_typeperiodegaec_id',\
                   'dd_datedebutactivite','dd_datefinactivite','kn_individu_national','type_contrat',\
                  'dn_entreprise_id','dd_datedebut','kc_resultatsbeneficiaire_id','date_fin_observation',\
                   'dn_horairehebdosalairie']

pr00_ppx005_drvder_periodeactivitegaec_keep = pr00_ppx005_drvder_periodeactivitegaec[colomns_to_keep]
pr00_ppx005_drvder_periodeactivitegaec_keep['source'] = 'GAEC'
pr00_ppx005_drvder_periodeactivitegaec_keep['siret'] = np.nan
pr00_ppx005_drvder_periodeactivitegaec_keep['nir'] = np.nan

pr00_ppx005_drvder_periodeactivitegaec_keep['idendifiant'] = pr00_ppx005_drvder_periodeactivitegaec_keep[['kn_periodeactivitegaec','dc_origineinformationgaec_id','dc_typeperiodegaec_id']].apply(lambda x:{'kn_periodeactivitegaec':x[0],'dc_origineinformationgaec_id':x[1],'dc_typeperiodegaec_id':x[2]},axis=1)
pr00_ppx005_drvder_periodeactivitegaec_keep['quantite_travaille'] = pr00_ppx005_drvder_periodeactivitegaec_keep['dn_horairehebdosalairie'].apply(lambda x:{'dn_horairehebdosalairie':x})


colomns_to_keep = ['source','siret','dn_entreprise_id','nir','kn_individu_national',\
                   'type_contrat', 'dd_datedebutactivite','dd_datefinactivite','idendifiant',\
                   'quantite_travaille','dd_datedebut',\
                   'kc_resultatsbeneficiaire_id', 'date_fin_observation']

pr00_ppx005_drvder_periodeactivitegaec_keep = pr00_ppx005_drvder_periodeactivitegaec_keep[colomns_to_keep]

columns_to_rename = {'siret':'siret','dn_entreprise_id':'BNE','nir':'nir','kn_individu_national':'BNI',\
                    'dd_datedebutactivite':'datedebucontrat','dd_datefinactivite':'datefincontrat'}

pr00_ppx005_drvder_periodeactivitegaec_keep = pr00_ppx005_drvder_periodeactivitegaec_keep.rename(columns=columns_to_rename)

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_all/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [129]:
import pandas as pd
contrat_prestation = pd.concat([pr00_ppx005_drvder_periodeactivitegaec_keep,pr00_ppx005_xdpdpa_dpae_keep,\
                               pr00_pdsnpe_declaractivitesal_keep],axis=0)


resultat = {'VA2':"SORTIE ANTICIPEE/ABANDON A PARTIR DU 2EME MOIS",\
            'VAB':"SORTIE ANTICIPEE/ABANDON AVANT LE 2EME MOIS",\
            'VAT':"A SUIVI LA PRESTATION DANS SA TOTALITE",\
            'VED':"REPRISE D'EMPLOI CDD OU MISSION INTERIM DE 3 A 6 MOIS",\
            'VEM':"REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM >= A 6 MOIS"}

contrat_prestation['resultat_string'] = contrat_prestation['kc_resultatsbeneficiaire_id'].apply(lambda x: resultat[x])
contrat_prestation

,source,siret,BNE,nir,BNI,type_contrat,datedebucontrat,datefincontrat,idendifiant,quantite_travaille,dd_datedebut,kc_resultatsbeneficiaire_id,date_fin_observation,resultat_string
247,GAEC,NaN,30786985.0,NaN,1287246151,CDD,2019-10-10,2019-11-06 00:00:00,"{'kn_periodeactivitegaec': 1644124963176472, '...",{'dn_horairehebdosalairie': 37.92},2019-08-14 10:30:00,VAT,2020-01-11 10:30:00,A SUIVI LA PRESTATION DANS SA TOTALITE
248,GAEC,NaN,30786985.0,NaN,1287246151,CDD,2019-12-05,2019-12-24 00:00:00,"{'kn_periodeactivitegaec': 1681969928355864, '...",{'dn_horairehebdosalairie': 37.92},2019-08-14 10:30:00,VAT,2020-01-11 10:30:00,A SUIVI LA PRESTATION DANS SA TOTALITE
249,GAEC,NaN,30786985.0,NaN,1287246151,CDD,2019-11-07,2019-12-04 00:00:00,"{'kn_periodeactivitegaec': 1668055207378968, '...",{'dn_horairehebdosalairie': 37.92},2019-08-14 10:30:00,VAT,2020-01-11 10:30:00,A SUIVI LA PRESTATION DANS SA TOTALITE
488,GAEC,NaN,1407465.0,NaN,1289201906,CDD,2019-09-06,2019-09-29 00:00:00,"{'kn_periodeactivitegaec': 1610607264022552, '...",{'dn_horairehebdosalairie': 37.92},2019-08-20 10:30:00,VAT,2020-01-17 10:30:00,A SUIVI LA PRESTATION DANS SA TOTALITE
490,GAEC,NaN,31410110.0,NaN,1289201906,CTT,2019-10-28,2019-11-16 00:00:00,"{'kn_periodeactivitegaec': 1677855555534872, '...",{'dn_horairehebdosalairie': 35.0},2019-08-20 10:30:00,VAT,2020-01-17 10:30:00,A SUIVI LA PRESTATION DANS SA TOTALITE
580,GAEC,NaN,37554300.0,NaN,1073397566,NR,2019-09-16,2019-12-22 00:00:00,"{'kn_periodeactivitegaec': 1681975858102296, '...",{'dn_horairehebdosalairie': 43.33},2019-08-21 10:00:00,VEM,2020-01-18 10:00:00,REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM ...
583,GAEC,NaN,1747156.0,NaN,1184195565,NR,2019-09-09,2019-12-13 00:00:00,"{'kn_periodeactivitegaec': 1678266413383704, '...",{'dn_horairehebdosalairie': 37.92},2019-08-19 09:00:00,VEM,2020-01-16 09:00:00,REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM ...
759,GAEC,NaN,989568.0,NaN,1379732579,CDD,2019-09-11,2019-10-31 00:00:00,"{'kn_periodeactivitegaec': 1638314830217240, '...",{'dn_horairehebdosalairie': 37.92},2019-08-28 09:00:00,VAT,2020-01-25 09:00:00,A SUIVI LA PRESTATION DANS SA TOTALITE
765,GAEC,NaN,989568.0,NaN,1379732579,CDD,2019-11-25,2019-12-20 00:00:00,"{'kn_periodeactivitegaec': 1681363809812504, '...",{'dn_horairehebdosalairie': 37.92},2019-08-28 09:00:00,VAT,2020-01-25 09:00:00,A SUIVI LA PRESTATION DANS SA TOTALITE
774,GAEC,NaN,22212908.0,NaN,2022779058,NR,2019-08-19,2019-09-13 00:00:00,"{'kn_periodeactivitegaec': 1601990393479192, '...",{'dn_horairehebdosalairie': 5.39},2019-08-14 09:30:00,VEM,2020-01-11 09:30:00,REPRISE D'EMPLOI CDI / CDD OU MISSION INTERIM ...


In [161]:
#contrat_prestation['datefincontrat']
#contrat_prestation['datefincontrat'].apply(lambda x:np.datetime64('NaT') if x==np.datetime64('NaT')   else datetime.strptime(str(x)[0:10], "%Y-%m-%d"))

contrat_prestation['dat_new'] = contrat_prestation['datefincontrat'].apply(lambda x :'NR' if str(x)=='NaT' else str(x)[0:10])
contrat_prestation['dat_new'].apply(lambda x:np.datetime64('NaT') if x=='NR'   else datetime.strptime(x, "%Y-%m-%d"))

ValueError: time data 'nan' does not match format '%Y-%m-%d'

In [ ]:
#contrat_prestation.loc[contrat_prestation['datefincontrat']==np.datetime64('NaT')]
contrat_prestation['dat_new'] = contrat_prestation['datefincontrat'].apply(lambda x :'NR' if str(x)=='NaT' else str(x)[0:10])
contrat_prestation['dat_new'].apply(lambda x:np.datetime64('NaT') if x=='NR'   else datetime.strptime(x, "%Y-%m-%d"))


In [245]:
datetime(datetime(2020,5,21,0,9,7,3).date().year,datetime(2020,5,21,0,9,7,3).date().month,1)

datetime.datetime(2020, 5, 1, 0, 0)

In [246]:
datetime(2020,5,21)

datetime.datetime(2020, 5, 21, 0, 0)